In [1]:
import numpy as np
import pandas as pd

In [2]:
path = '/Users/zhuangyuhao/PycharmProjects/AC_model/'

dev = pd.read_excel(path + '结果测试_230331/会计上市主体数据检查_230407.xlsx')
check = pd.read_excel(path + '整合事务所模型_20220919/tmp2/上市公司表_230403.xlsx')

In [3]:
dev.drop(index=0, inplace=True)
dev.drop(columns=['describe', 'priority', 'stock_name', 'factor_id'], inplace=True)
dev['final_value'] = np.where(dev['final_value'] == '-999999.9999', 0, dev['final_value'])
dev = dev[dev['code'] != 'AF_CYD_lastfirm']

In [4]:
check_melted = check.melt(id_vars=['AF_CYD_factor01','AF_CYD_factor04'],
                          value_vars=check.columns[2:],
                          var_name='indicator_code',
                          value_name='indicator_value_check')
check_melted.rename(columns={'AF_CYD_factor01': 'stock_code',
                             'AF_CYD_factor04': 'year',
                             'indicator_code': 'code'},
                    inplace=True)

In [5]:
check_result = dev.merge(check_melted, on=['stock_code', 'year', 'code'], how='left')
check_result

,entity_id,entity_name,stock_code,name,code,year,final_value,id,indicator_value_check
0,1,平安银行股份有限公司,000001.SZ,审计费用变动率,AF_CYD_FEEGROWTH,2021,-0.25,3002262,-0.247507
1,1,平安银行股份有限公司,000001.SZ,审计费用是否大幅变动,AF_CYD_IF_FEECHANGE,2021,0,3002272,0
2,1,平安银行股份有限公司,000001.SZ,本年度是否未发生定增,AF_CYD_IF_PLACEMENT,2021,0,3002291,0
3,1,平安银行股份有限公司,000001.SZ,本年度是否未发生重组,AF_CYD_IF_REORGANIZATION,2021,0,3002306,0
4,1,平安银行股份有限公司,000001.SZ,本年度是否换所,AF_CYD_IF_FIRMCHANGE,2021,1,3002329,1.0
...,...,...,...,...,...,...,...,...,...
388498,31863,浙江海正生物材料股份有限公司,688203.SH,上市公司审计收费分行业占比,AF_factor50,2019,0,865427,NaN
388499,31863,浙江海正生物材料股份有限公司,688203.SH,审计费用,AF_CYD_IF_FEEVALUE,2019,0,3406375,NaN
388500,31863,浙江海正生物材料股份有限公司,688203.SH,是否存在会计差错更正,AF_correction_error,2019,NaN,3406828,NaN
388501,31863,浙江海正生物材料股份有限公司,688203.SH,业务总收入,AF_id_income,2019,NaN,3406850,NaN


In [6]:
check_result['indicator_value_check'] = check_result['indicator_value_check'].astype(float)
check_result['final_value'] = check_result['final_value'].astype(float)

In [7]:
check_result['match'] = np.where((abs(check_result.final_value - check_result.indicator_value_check) < 0.01), 1, 0)

In [8]:
check_result_error = check_result[check_result['match'] == 0]
check_result_error.dropna(subset='indicator_value_check', inplace=True)

/var/folders/21/r5tywv_91vz_8s1tgrs4qpjm0000gn/T/ipykernel_16350/2434901196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_result_error.dropna(subset='indicator_value_check', inplace=True)


In [9]:
# check_result_error = check_result_error[check_result_error['name'] != '本年度是否未发生定增']  # 自己错
# check_result_error = check_result_error[check_result_error['name'] != '本年是否年审和内审事务所不同']  # 开发错

check_result_error = check_result_error[check_result_error['name'] != '是否该事务所前五大客户']  # 开发错
# check_result_error = check_result_error[check_result_error['name'] != '是否为严重问题公司']  # 开发错
# check_result_error = check_result_error[check_result_error['name'] != '是否为失信人']  # 开发错
# check_result_error = check_result_error[check_result_error['name'] != '本年是否违规']  # 开发错

In [10]:
check_result_error

,entity_id,entity_name,stock_code,name,code,year,final_value,id,indicator_value_check,match
17,1,平安银行股份有限公司,000001.SZ,本年是否年审和内审事务所不同,AF_CYD_IF_TWOAUDIT,2021,1.0,3002484,0.0,0
92,2,泛海控股股份有限公司,000046.SZ,本年是否年审和内审事务所不同,AF_CYD_IF_TWOAUDIT,2021,1.0,3002512,0.0,0
167,3,申万宏源集团股份有限公司,000166.SZ,本年是否年审和内审事务所不同,AF_CYD_IF_TWOAUDIT,2021,1.0,3002612,0.0,0
242,4,渤海租赁股份有限公司,000415.SZ,本年是否年审和内审事务所不同,AF_CYD_IF_TWOAUDIT,2021,1.0,3002682,0.0,0
317,5,民生控股股份有限公司,000416.SZ,本年是否年审和内审事务所不同,AF_CYD_IF_TWOAUDIT,2021,1.0,3002719,0.0,0
...,...,...,...,...,...,...,...,...,...,...
388130,31859,合肥新汇成微电子股份有限公司,688403.SH,本年是否年审和内审事务所不同,AF_CYD_IF_TWOAUDIT,2021,1.0,3002382,0.0,0
388208,31860,熵基科技股份有限公司,301330.SZ,本年是否年审和内审事务所不同,AF_CYD_IF_TWOAUDIT,2021,1.0,3002361,0.0,0
388286,31861,深圳市路维光电股份有限公司,688401.SH,本年是否年审和内审事务所不同,AF_CYD_IF_TWOAUDIT,2021,1.0,3002474,0.0,0
388364,31862,东莞市凯格精机股份有限公司,301338.SZ,本年是否年审和内审事务所不同,AF_CYD_IF_TWOAUDIT,2021,1.0,3002501,0.0,0


In [11]:
check_result_error.to_excel(path + '结果测试_230331/测试结果_上市企业_error_230407.xlsx', index=False)
